# Visualising Emissions

It is often useful to be able to produce visualisations of profiling data, including CO~2~ estimates.

Here we will use the [Weights & Biases](https://wandb.ai) platform to construct a simple profiling dashboard for our code.

### Step 1: 
Visit https://wandb.ai/authorize

### Step 2: 
Log in with your GitHub account or with Google

### Step 3: 
Copy API key (under the profile dropdown on the right-hand side)

### Step 4: 
Log in to W&B 

- In VSCode Terminal, run python3 (or python)

- `import wandb`

- `wandb.login()`

- Enter your choice: 2

- Paste the copied API key


### Step 5: 
Exit python environment from terminal

### Step 6: 
Run your script with `wandb.init()` and then `wandb.log`, and `wandb.finish()` to end the wandb run.

In [12]:
import numpy as np
import psutil, time, os, gc, warnings
from codecarbon import EmissionsTracker
import wandb

warnings.filterwarnings('ignore')


# Function to get memory usage
def get_memory_usage():
    process = psutil.Process(os.getpid())
    mem_info = process.memory_info()
    return mem_info.rss / 1024 / 1024  # Memory in MB

# Function to get CPU usage
def get_cpu_usage():
    return psutil.cpu_percent(interval=1)

# Function to get run stats with multiple iterations
def run_stats(project_name, analysis_func, num_runs):
    results = []
    emissions = []
    runtimes = []
    cpu_usages = []
    mem_usages = []
    
    # Initialize wandb run
    wandb.init(project=project_name, reinit=True)
    
    for run_idx in range(num_runs):

        gc.collect()  # Clear memory before each run
        tracker = EmissionsTracker()
        tracker.start()
        
        start_mem = get_memory_usage()
        start_cpu = get_cpu_usage()
        start_time = time.time()
        
        result = analysis_func()
        
        end_time = time.time()
        end_cpu = get_cpu_usage()
        end_mem = get_memory_usage()
       
        emission = tracker.stop()

        results.append(result)

        emissions.append(emission)
        cpu_usages.append((start_cpu + end_cpu) / 2)  # Average CPU usage during run
        mem_usages.append(end_mem - start_mem)
        runtimes.append(end_time - start_time)

        
        # Log per-run metrics to wandb
        wandb.log({
            "Runtime (s)": runtimes[-1],
            "CPU Usage (%)": cpu_usages[-1],
            "RAM Usage (M)": mem_usages[-1],
            "Emissions (kgCO2)": emission,
        })

        # Print result
        print(result)

    # End wandb run
    wandb.finish()  
    
    # Compute statistics
    metrics = {
        "mean runtime (s)": np.mean(runtimes),
        "std runtime (s)": np.std(runtimes) if num_runs > 1 else 0,
        "mean RAM (M)": np.mean(mem_usages),
        "std RAM (M)": np.std(mem_usages) if num_runs > 1 else 0,
        "mean CPU (%)": np.mean(cpu_usages),
        "std CPU (%)": np.std(cpu_usages) if num_runs > 1 else 0,
        "mean emissions (kgCO2)": np.mean(emissions),
        "std emissions (kgCO2)": np.std(emissions) if num_runs > 1 else 0,
    }
    
    return (results, metrics)

In [14]:
def fib(n):
    if(n == 0):
        return 0
    if(n == 1):
        return 1
    return fib(n - 1) + fib(n - 2)

def fibs(n):
    return [fib(i) for i in range(n)]

def fib_task():
    return fibs(35)

(results, metrics) = run_stats("fib", fib_task, 10)

print(results[0])

for k in metrics.keys():
    print(f"{k}: {metrics[k]}")

[codecarbon INFO @ 15:26:53] Codecarbon is taking the configuration from the local file /Users/jpinney/Documents/GitHub/ECRI-RCDS-Hackathon/.codecarbon.config
[codecarbon WARNING @ 15:26:53] Multiple instances of codecarbon are allowed to run at the same time.
[codecarbon WARNING @ 15:26:53] Error while trying to count physical CPUs: [Errno 2] No such file or directory: 'lscpu'. Defaulting to 1.
[codecarbon INFO @ 15:26:53] [setup] RAM Tracking...
[codecarbon INFO @ 15:26:53] [setup] CPU Tracking...
[codecarbon WARNING @ 15:26:53] We saw that you have a Apple M3 but we don't know it. Please contact us.
[codecarbon DEBUG @ 15:26:53] is_psutil_available() : psutil.cpu_times().nice is too small : 0.0 !
[codecarbon DEBUG @ 15:26:53] Not using PowerGadget, an exception occurred while instantiating IntelPowerGadget : Intel Power Gadget executable not found on darwin
[codecarbon DEBUG @ 15:26:53] Not using the RAPL interface, an exception occurred while instantiating IntelRAPL : Platform not 

[0, 1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377, 610, 987, 1597, 2584, 4181, 6765, 10946, 17711, 28657, 46368, 75025, 121393, 196418, 317811, 514229, 832040, 1346269, 2178309, 3524578, 5702887]


[codecarbon WARNING @ 15:27:01] No CPU tracking mode found. Falling back on estimation based on TDP for CPU. 
 Mac OS and ARM processor detected: Please enable PowerMetrics sudo to measure CPU

[codecarbon INFO @ 15:27:01] CPU Model on constant consumption mode: Apple M3
[codecarbon DEBUG @ 15:27:01] is_psutil_available() : psutil.cpu_times().nice is too small : 0.0 !
[codecarbon WARNING @ 15:27:01] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon INFO @ 15:27:01] [setup] GPU Tracking...
[codecarbon INFO @ 15:27:01] No GPU found.
[codecarbon INFO @ 15:27:01] The below tracking methods have been set up:
                RAM Tracking Method: RAM power estimation model
                CPU Tracking Method: global constant
                GPU Tracking Method: Unspecified
            
[codecarbon INFO @ 15:27:01] >>> Tracker's metadata:
[codecarbon INFO @ 15:27:01]   Platform system: macOS-13.7.6-arm64-arm-64bit-Mach-O
[codecarbon INFO @ 15:27:01]   Python version: 3

[0, 1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377, 610, 987, 1597, 2584, 4181, 6765, 10946, 17711, 28657, 46368, 75025, 121393, 196418, 317811, 514229, 832040, 1346269, 2178309, 3524578, 5702887]


[codecarbon DEBUG @ 15:27:06] Not using PowerMetrics, sudo password prompt detected.
                    If you want to enable Powermetrics please modify your sudoers file
                    as described in :
                    https://mlco2.github.io/codecarbon/methodology.html#power-usage
                
[codecarbon WARNING @ 15:27:06] No CPU tracking mode found. Falling back on estimation based on TDP for CPU. 
 Mac OS and ARM processor detected: Please enable PowerMetrics sudo to measure CPU

[codecarbon INFO @ 15:27:06] CPU Model on constant consumption mode: Apple M3
[codecarbon DEBUG @ 15:27:06] is_psutil_available() : psutil.cpu_times().nice is too small : 0.0 !
[codecarbon WARNING @ 15:27:06] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon INFO @ 15:27:06] [setup] GPU Tracking...
[codecarbon INFO @ 15:27:06] No GPU found.
[codecarbon INFO @ 15:27:06] The below tracking methods have been set up:
                RAM Tracking Method: RAM power estima

...

[0, 1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377, 610, 987, 1597, 2584, 4181, 6765, 10946, 17711, 28657, 46368, 75025, 121393, 196418, 317811, 514229, 832040, 1346269, 2178309, 3524578, 5702887]


[codecarbon WARNING @ 15:27:43] No CPU tracking mode found. Falling back on estimation based on TDP for CPU. 
 Mac OS and ARM processor detected: Please enable PowerMetrics sudo to measure CPU

[codecarbon INFO @ 15:27:43] CPU Model on constant consumption mode: Apple M3
[codecarbon DEBUG @ 15:27:43] is_psutil_available() : psutil.cpu_times().nice is too small : 0.0 !
[codecarbon WARNING @ 15:27:43] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon INFO @ 15:27:43] [setup] GPU Tracking...
[codecarbon INFO @ 15:27:43] No GPU found.
[codecarbon INFO @ 15:27:43] The below tracking methods have been set up:
                RAM Tracking Method: RAM power estimation model
                CPU Tracking Method: global constant
                GPU Tracking Method: Unspecified
            
[codecarbon INFO @ 15:27:43] >>> Tracker's metadata:
[codecarbon INFO @ 15:27:43]   Platform system: macOS-13.7.6-arm64-arm-64bit-Mach-O
[codecarbon INFO @ 15:27:43]   Python version: 3

[0, 1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377, 610, 987, 1597, 2584, 4181, 6765, 10946, 17711, 28657, 46368, 75025, 121393, 196418, 317811, 514229, 832040, 1346269, 2178309, 3524578, 5702887]


CPU Usage (%),▅▇▂▃█▄▂▅▁▄
Emissions (kgCO2),▁▅▁▁▄▃▄█▁▅
RAM Usage (M),█▁██▆██▁██
Runtime (s),▁▅▁▁▅▃▄█▁▅
CPU Usage (%),20.75
Emissions (kgCO2),2e-05
RAM Usage (M),-2.23438
Runtime (s),1.80506


[0, 1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377, 610, 987, 1597, 2584, 4181, 6765, 10946, 17711, 28657, 46368, 75025, 121393, 196418, 317811, 514229, 832040, 1346269, 2178309, 3524578, 5702887]
mean runtime (s): 1.7732422828674317
std runtime (s): 0.04396452325661924
mean RAM (M): -25.94375
std RAM (M): 44.79938146981636
mean CPU (%): 20.335
std CPU (%): 3.716924131590527
mean emissions (kgCO2): 1.9946257233078325e-05
std emissions (kgCO2): 2.3069834261236267e-07


## Task: C0~2~ visualisation

Try making a W&B dashboard for your pathfinding code.

---